In [1]:
pattern = "*_DW.mrc"
output_path = "Position_9.st"

In [2]:
import glob
import numpy as np
import mrcfile
from pathlib import Path
import os

In [3]:
def get_image_paths(pattern = "*_DW.mrc"):
    paths = []
    for p in glob.glob(pattern):
        paths.append(Path(p))
    return paths

def get_angles_from_paths(paths):
    angles = []
    for p in paths:
        split0 = str(p).split("[")
        angle = float(split0[1].split("]")[0])
        angles.append(angle)
    return angles

def make_angle_path_dict(angles, paths):
    ang_path_dict = dict(zip(angles, paths))
    sorted_dict = dict(sorted(ang_path_dict.items()))
    return sorted_dict
def read_mrc(path):
    with mrcfile.open(path, permissive=True) as mrc:
        data = mrc.data
        voxel_size = mrc.voxel_size
    return data, voxel_size
def make_stack_from_ang_path_dict(ang_path_dict):
    stack = []
    for ang, path in ang_path_dict.items():
        data, voxel_size = read_mrc(path)
        stack.append(data)
    return np.array(stack), voxel_size
def save_stack(file_path, stack, voxel_size, angles=None):
    stack_out_path = Path(file_path)
    with mrcfile.new(stack_out_path, overwrite = True) as mrc:
        mrc.set_data(stack)
        mrc.set_image_stack()
        mrc.voxel_size = voxel_size
    if angles:
        ang_out_path = stack_out_path.parent / (stack_out_path.stem + '.ang')
        with open(ang_out_path, 'w') as f:
            for ang in angles:
                print(ang, file=f)

In [4]:
paths = get_image_paths()
angles = get_angles_from_paths(paths)
ang_path_dict = make_angle_path_dict(angles, paths)
stack, voxel_size = make_stack_from_ang_path_dict(ang_path_dict)

C:\Users\ib19e403\AppData\Local\Continuum\anaconda3\envs\image\lib\site-packages\mrcfile\mrcinterpreter.py:208: RuntimeWarning: Map ID string not found - not an MRC file, or file is corrupt
  warnings.warn(msg, RuntimeWarning)


In [ ]:
save_stack(output_path, stack, voxel_size, angles=angles)


In [ ]:
os.system(f"3dmod {output_path}")